In [2]:
# Importar bibliotecas necessárias
import os
import numpy as np
import pandas as pd
from PIL import Image

# Carregar o arquivo CSV de treino
df = pd.read_csv('dataset/train.csv')
print("Colunas do CSV:", df.columns.tolist())
print(df.head(5))  # Exibe as primeiras 5 linhas para inspecionar


Colunas do CSV: ['Unnamed: 0', 'original_path', 'id', 'label', 'label_str', 'path']
   Unnamed: 0                                      original_path     id  \
0        1052  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  28609   
1       13406  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  30395   
2       16376  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  22430   
3       31373  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  58380   
4       21034  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  62044   

   label label_str                  path  
0      1      real  train/real/28609.jpg  
1      1      real  train/real/30395.jpg  
2      1      real  train/real/22430.jpg  
3      1      real  train/real/58380.jpg  
4      1      real  train/real/62044.jpg  


In [7]:
print(df.columns)

Index(['Unnamed: 0', 'original_path', 'id', 'label', 'label_str', 'path'], dtype='object')


In [11]:
base_path = 'dataset/imagens'  # ajuste se a raiz das imagens for diferente

for idx, row in df.iterrows():
    nome_arquivo = row['path']        # nome relativo do arquivo, tipo "fake/image123.jpg"
    rotulo = row['label_str']         # "REAL" ou "FAKE" — útil se quiser classificar depois

    # Caminho completo do arquivo de imagem
    caminho_completo = os.path.join(base_path, nome_arquivo)

    try:
        img = Image.open(caminho_completo).convert('RGB')
        img = img.resize((64, 64))
        img_array = np.array(img) / 255.0
        imagens.append(img_array)
        labels.append(rotulo)
    except Exception as e:
        print(f"Erro ao carregar imagem {caminho_completo}: {e}")


In [12]:
# Converter a lista de imagens para um array numpy
imagens = np.array(imagens, dtype=np.float32)
print("Total de imagens carregadas:", imagens.shape[0])
print("Dimensão de cada imagem:", imagens.shape[1:], "(altura, largura, canais)")

Total de imagens carregadas: 7000
Dimensão de cada imagem: (64, 64, 3) (altura, largura, canais)


In [13]:
import tensorflow as tf

# Definir parâmetros de treino
BUFFER_SIZE = imagens.shape[0]  # tamanho do conjunto (para embaralhamento)
BATCH_SIZE = 32                 # tamanho do lote (você pode ajustar; 32 é um bom começo)

# Criar dataset TensorFlow embaralhado e em batched
dataset = tf.data.Dataset.from_tensor_slices(imagens)
dataset = dataset.shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE)

# Definir dimensão do vetor de entrada (ruído) para o gerador
latent_dim = 100  # dimensão do vetor aleatório que o gerador recebe (100 é padrão comum)


In [14]:
from tensorflow.keras import layers

def build_generator(latent_dim=100):
    model = tf.keras.Sequential()
    # 1. Camada densa inicial para gerar uma ativação base
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.BatchNormalization())  # normalização para estabilizar treino
    model.add(layers.LeakyReLU())           # ativação LeakyReLU para não saturar gradientes

    model.add(layers.Reshape((8, 8, 256)))  # reshape para mapa de 8x8 com 256 filtros
    # Agora a forma do tensor é (8, 8, 256). Vamos realizar upsampling sucessivo:

    # 2. Camada de deconvolução 1: 8x8 -> 16x16
    model.add(layers.Conv2DTranspose(128, kernel_size=(5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    # Agora a saída é (16, 16, 128)

    # 3. Camada de deconvolução 2: 16x16 -> 32x32
    model.add(layers.Conv2DTranspose(64, kernel_size=(5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    # Saída agora: (32, 32, 64)

    # 4. Camada de deconvolução 3: 32x32 -> 64x64 (camada de saída)
    model.add(layers.Conv2DTranspose(3, kernel_size=(5, 5), strides=(2, 2), padding='same', use_bias=False, activation='sigmoid'))
    # Saída final esperada: (64, 64, 3) com valores entre 0 e 1 (devido à sigmoide)

    return model

# Construir o gerador e exibir um resumo da arquitetura
generator = build_generator(latent_dim)
generator.summary()


c:\Users\madar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16384)          │     1,638,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16384)          │        65,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 128)    │       819,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 3)      │         4,800 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,733,504 (10.43 MB)

 Trainable params: 2,700,352 (10.30 MB)

 Non-trainable params: 33,152 (129.50 KB)

In [15]:
def build_discriminator():
    model = tf.keras.Sequential()
    # 1. Camada de convolução 1: reduz 64x64 -> 32x32
    model.add(layers.Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=(64, 64, 3)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    # Saída: (32, 32, 64)

    # 2. Camada de convolução 2: 32x32 -> 16x16
    model.add(layers.Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    # Saída: (16, 16, 128)

    # 3. Camada de convolução 3: 16x16 -> 8x8
    model.add(layers.Conv2D(256, kernel_size=(5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    # Saída: (8, 8, 256)

    # 4. Saída: achatar e camada densa sigmoid
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    # Resultado: um valor entre 0 e 1 indicando se a imagem é "real" (1) ou "fake" (0)

    return model

# Construir o discriminador e exibir resumo
discriminator = build_discriminator()
discriminator.summary()


c:\Users\madar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 256)      │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │        16,385 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,045,633 (3.99 MB)

 Trainable params: 1,045,633 (3.99 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Função de loss e otimizadores
loss_fn = tf.keras.losses.BinaryCrossentropy()  # usa cross-entropia binária (discriminator output é sigmoid)
optimizer_gen = tf.keras.optimizers.Adam(learning_rate=0.0002)
optimizer_disc = tf.keras.optimizers.Adam(learning_rate=0.0002)

In [ ]:
# Parâmetros de treinamento
EPOCHS = 4  # número de épocas (pode aumentar se quiser resultados melhores, mas leva mais tempo)
seed = tf.random.normal([16, latent_dim])
# seed: um conjunto fixo de vetores aleatórios para gerar imagens de exemplo consistentemente a cada época (opcional, para acompanhamento visual)

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}/{EPOCHS}')
    for real_images in dataset:
        batch_size = real_images.shape[0]

        # Treinamento do Discriminador:
        # Gerar um lote de imagens falsas
        noise = tf.random.normal([batch_size, latent_dim])
        fake_images = generator(noise, training=True)

        # Labels para treino do disc: 1 para reais, 0 para falsas
        real_labels = tf.ones((batch_size, 1))
        fake_labels = tf.zeros((batch_size, 1))

        # Calcular perda do discriminador (em um contexto de gravação de gradientes)
        with tf.GradientTape() as tape_disc:
            # predições do disc para reais e falsas
            pred_real = discriminator(real_images, training=True)
            pred_fake = discriminator(fake_images, training=True)
            # calcula loss para ambos
            loss_real = loss_fn(real_labels, pred_real)
            loss_fake = loss_fn(fake_labels, pred_fake)
            loss_disc = (loss_real + loss_fake) / 2.0  # perda total do disc (média das duas)
        # calcular gradientes e aplicar ao discriminador
        grads_disc = tape_disc.gradient(loss_disc, discriminator.trainable_variables)
        optimizer_disc.apply_gradients(zip(grads_disc, discriminator.trainable_variables))

        # Treinamento do Gerador:
        # Gerar novo lote de imagens falsas (usando o gerador atualizado? - aqui gerador ainda não foi atualizado na iteração atual)
        noise = tf.random.normal([batch_size, latent_dim])
        # Nota: não reutilizamos fake_images anterior para evitar que o gerador treine no mesmo input (poderia reutilizar, mas é comum usar novo ruído)
        with tf.GradientTape() as tape_gen:
            fake_images = generator(noise, training=True)
            pred_fake = discriminator(fake_images, training=False)
            # pred_fake com training=False para não atualizar o disc aqui
            loss_gen = loss_fn(tf.ones((batch_size, 1)), pred_fake)
            # queremos que o disc acredite que as imagens falsas são reais (rótulo 1), por isso tf.ones(...)
        grads_gen = tape_gen.gradient(loss_gen, generator.trainable_variables)
        optimizer_gen.apply_gradients(zip(grads_gen, generator.trainable_variables))
    # Fim do loop de batches de uma época

    # Exibir perdas médias da última batch processada da época (só para acompanhamento)
    print(f"-> Loss Disc: {loss_disc:.4f} | Loss Gen: {loss_gen:.4f}")

    # (Opcional) Gerar imagens de exemplo a cada N épocas para monitorar visualmente
    if epoch % 10 == 0 or epoch == 1:
        sample_noise = tf.random.normal([1, latent_dim])
        generated_img = generator(sample_noise, training=False)[0].numpy()
        img_show = (generated_img * 255).astype('uint8')  # converte para uint8 para salvar/visualizar
        Image.fromarray(img_show).save(f"exemplo_epoca_{epoch}.png")  # salva uma imagem gerada (exemplo)
        print(f"[Imagem de exemplo salva: exemplo_epoca_{epoch}.png]")


Epoch 1/4
-> Loss Disc: 0.1627 | Loss Gen: 2.8913
[Imagem de exemplo salva: exemplo_epoca_1.png]
Epoch 2/4
-> Loss Disc: 0.1780 | Loss Gen: 2.2163
Epoch 3/4
-> Loss Disc: 0.5439 | Loss Gen: 2.6033
Epoch 4/4
-> Loss Disc: 0.0512 | Loss Gen: 9.3909


In [25]:
# Pressupondo que o modelo discriminador (por exemplo, uma rede Keras já treinada)
# está disponível na variável `discriminator`.
# Salvamos o modelo treinado no disco com o nome especificado.
discriminator.save("discriminador_gan_rvf10k.h5")
print("Modelo discriminador salvo em discriminador_gan_rvf10k.h5")


Modelo discriminador salvo em discriminador_gan_rvf10k.h5


In [ ]:
# Carrega o modelo discriminador salvo do arquivo, sem precisar re-treinar.
from tensorflow.keras.models import load_model

discriminator = load_model("discriminador_gan_rvf10k.h5")
print("Modelo discriminador carregado com sucesso.")

In [29]:
import os
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Diretórios contendo as imagens reais e falsas de validação
dir_real = "dataset/valid/real"
dir_fake = "dataset/valid/fake"

# Listas para armazenar rótulos verdadeiros e predições do modelo
y_true = []   # rótulos reais (1 para REAL, 0 para FAKE)
y_pred = []   # rótulos previstos pelo modelo (1 ou 0)
y_prob = []   # probabilidade estimada de ser REAL (saída do discriminador)

# Processa todas as imagens REAIS de validação
for filename in os.listdir(dir_real):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # garante arquivo de imagem
        img_path = os.path.join(dir_real, filename)
        image = Image.open(img_path)
        # Redimensiona para 64x64 pixels e converte para RGB
        image = image.resize((64, 64)).convert('RGB')
        # Normaliza os pixels para o intervalo [0, 1]
        img_array = np.array(image).astype('float32') / 255.0
        # Adiciona dimensão batch (1, 64, 64, 3) antes de alimentar o modelo
        img_array = np.expand_dims(img_array, axis=0)
        # Realiza a predição com o discriminador carregado
        prob_real = discriminator.predict(img_array)[0][0]  # probabilidade de ser "REAL"
        pred_label = 1 if prob_real >= 0.5 else 0           # classifica como 1 (REAL) se prob >= 0.5, senão 0 (FAKE)
        # Armazena o rótulo verdadeiro e as predições
        y_true.append(1)           # rótulo verdadeiro 1 (REAL)
        y_pred.append(pred_label)  # rótulo previsto pelo modelo
        y_prob.append(prob_real)   # probabilidade estimada de ser REAL

# Processa todas as imagens FALSAS de validação
for filename in os.listdir(dir_fake):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(dir_fake, filename)
        image = Image.open(img_path)
        image = image.resize((64, 64)).convert('RGB')
        img_array = np.array(image).astype('float32') / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        prob_real = discriminator.predict(img_array)[0][0]
        pred_label = 1 if prob_real >= 0.5 else 0
        y_true.append(0)           # rótulo verdadeiro 0 (FAKE)
        y_pred.append(pred_label)
        y_prob.append(prob_real)

# Converte as listas de resultados para arrays do NumPy
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calcula as métricas de desempenho do discriminador
acuracia = accuracy_score(y_true, y_pred)
precisao = precision_score(y_true, y_pred)
revocacao = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Exibe as métricas principais de desempenho
print(f"Acurácia: {acuracia:.4f}")
print(f"Precisão: {precisao:.4f}")
print(f"Recall (Revocação): {revocacao:.4f}")
print(f"F1-score: {f1:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━

In [32]:
import random
import os
from PIL import Image
import numpy as np

# Caminho base do conjunto de validação
base_dir = "dataset/valid"
subpastas = ["real", "fake"]

# Escolher aleatoriamente a subpasta e uma imagem
subpasta_escolhida = random.choice(subpastas)
caminho_subpasta = os.path.join(base_dir, subpasta_escolhida)
lista_arquivos = [f for f in os.listdir(caminho_subpasta) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if not lista_arquivos:
    print(f"Nenhuma imagem encontrada na pasta {caminho_subpasta}.")
else:
    imagem_escolhida = random.choice(lista_arquivos)
    caminho_imagem = os.path.join(caminho_subpasta, imagem_escolhida)

    # Carregar e preparar a imagem
    img = Image.open(caminho_imagem).convert("RGB").resize((64, 64))
    img_array = np.array(img).astype("float32") / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Fazer a predição
    prob_real = discriminator.predict(img_array)[0][0]
    rotulo_predito = "REAL" if prob_real >= 0.5 else "FAKE"

    # Exibir resultados
    print(f"Imagem escolhida: {imagem_escolhida} ({subpasta_escolhida.upper()})")
    print(f"Classificação do modelo: {rotulo_predito}")
    print(f"Probabilidade de ser REAL: {prob_real:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Imagem escolhida: JCDSPGEYHF.jpg (FAKE)
Classificação do modelo: REAL
Probabilidade de ser REAL: 0.9986
